#10K


In [ ]:
import torch
from torchvision import datasets, transforms
import time

# def distance_euclid(v1, v2):
#     return torch.pairwise_distance(v1, v2, p=2)


# def matrix_adjacent(dataset, sigma=1.0):
#     M = torch.zeros((len(dataset), len(dataset)))
#     for i in range(len(dataset)):
#         for j in range(i + 1, len(dataset)):
#             M[i][j] = torch.exp(-torch.sum(torch.pow(dataset[i] - dataset[j], 2)) / (sigma ** 2))
#             M[j][i] = M[i][j]
#     return M

def matrix_adjacent(dataset, sigma=1.0):
    n=len(dataset)
    ATA = torch.matmul(dataset,dataset.T)
    diag = torch.diag(ATA)
    norms = torch.matmul(torch.ones(1,n).cuda().T,diag[None,:])
    W = norms + norms.T - 2*ATA
    W = torch.exp(-1/sigma**2 * W)
    W-= torch.diag(W).diag()
    return W

def matrix_laplacian(ma):
    matrix_degree = torch.diag(torch.sum(ma, dim=0))
    ml = matrix_degree - ma
    return ml


if __name__ == "__main__":

    train_data = datasets.MNIST(root='./MNIST', train=True, download=True)
    test_data = datasets.MNIST(root='./MNIST', train=False, download=True)

    num_label = 1000
    num_unlabel = 10000

    dataset_label = test_data.data[:num_label].flatten(1).type(torch.float32)
    dataset_unlabel = train_data.data[:num_unlabel].flatten(1).type(torch.float32)
    dataset  = torch.cat([dataset_label,dataset_unlabel],dim=0).cuda()/255.0


    # fl = torch.zeros((num_label, 10))
    # for i in range(fl.shape[0]):
    #     fl[i][test_data[i][1]] = 1.0
    # fl = fl.cuda()
    fl = torch.nn.functional.one_hot(test_data.targets)[:num_label].type(torch.float32).cuda()

    t0=time.time()

    ma = matrix_adjacent(dataset, 1.5)
    ml = matrix_laplacian(ma).cuda()
    fu = torch.mm(-torch.inverse(ml[num_label:, num_label:]).cuda(), torch.mm(ml[num_label:, :num_label].cuda(), fl))
    fu=fu.cpu()

    print(time.time()-t0)

    labels_dataset_unlabel =torch.argmax(fu,dim=1)

    # counter = sum([1 if labels_dataset_unlabel[i] == train_data[i][1] else 0 for i in range(num_unlabel)])
    counter = sum(labels_dataset_unlabel == train_data.targets[:num_unlabel])
    print(counter / num_unlabel)


Extracting ./MNIST/MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/MNIST/raw


Extracting ./MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/MNIST/raw


Extracting ./MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/MNIST/raw




Extracting ./MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


1.0819201469421387
tensor(0.9107)


OLD:
7429.0953104496

0.9107

NEW:
1.0819201469421387

tensor(0.9107)

#60K
